<img align="right" style="max-width: 200px; height: auto" src="./fs_logo.png">

##  Lab 06 - Adversarial Deep Learning

Seminar Künstliche Intelligenz, Frankfurt School, 2022

Die Analysen des Seminars **Künstliche Intelligenz** des Zertifikatstudiengangs **Certified Audit Data Scientist (CADS)** basieren auf Jupyter Notebook. Anhand solcher Notebooks ist es möglich eine Vielzahl von Datenanalysen und statistischen Validierungen durchzuführen.

<img align="center" style="max-width: 700px" src="./lab_06_banner.png">

Im letzten Lab haben Sie die verschiedenen Elemente eines Unsupervised Deep Learning Workflow kennengelernt z.B. Datenaufbereitung, Modell Training und Modell Validierung. In diesem sechsten Lab werden wir Jupyter Notebook verwenden, um ein weiteres **Deep Learning basiertes Audit-Analyseverfahren** zu implementieren und anzuwenden.

Hierzu werden wir die im Seminar vorgestellten Adversarial Autoencoder Neural Networks (Ad-AENNs) anwenden um (1) zunächst die generativen Prozesse einer Finanzbuchhaltung und (2) anschliessend entsprechende Prozessanomalien zu detektieren. Im Gegensatz zu klassischen Autoencoder Netzen lernen AENNs, die Eingabedaten in eine niedrig-dimensionale Repräsentation zu **encodieren**.  Gleichzeitig lernt das AENN, die ursprünglichen Daten wieder aus der enkodierten Repräsentation zu **dekodieren**. 

Die dekodierten Daten, die in der Regel als **Rekonstruktion** bezeichnet werden, sollten eine grosse Ähnlichkeit zu den ursprünglichen **Eingabedaten** aufweisen. Die Buchungssätze, für welche eine erfolgreiche Rekonstruktion nur fehlerhaft gelingt müssen deshalb eine oder mehrere ungewöhnliche Eigenschaften aufweisen. Die nachstehende Abbildung zeigt einen Überblick über den Deep Learning Prozess bzw. die AENN Netzarchitektur, welche wir in diesem Lab implementieren.

<img align="center" style="max-width: 900px" src="./process.png">

Im Rahmen des Lab werden wir wieder einige Funktionen der `PyTorch` Bibliothek nutzen, um das AENN zu implementieren und zu trainieren. Im Laufe des Trainingsprozess soll das AENN die charakteristische Eigenschaften historischer **Buchungen** bzw. **Journal Entries** lernen. Nach erfolgreichen Modelltraining, werden wir das Modell anwenden, um anhand des Rekonstruktionsfehlers ungewöhnliche Buchungen innerhalb des Datensatzes zu detektieren. Abschliessend werden wir die gelernten **Repräsentationen** der einzelnen Journaleinträge dazu verwenden, um die erhaltenen Ergebnisse noch aussagekräftiger zu interpretieren.

Bei etwaigen Fragen wenden Sie sich, wie immer gerne an uns via **marco (dot) schreyer (at) unisg (dot) ch**. Wir wünschen Ihnen Viel Freude mit unseren Notebooks und Ihren revisorischen Analysen!

## Lernziele des Labs:

Nach der heutigen Übung sollten Sie in der Lage sein:

>1. Die **Grundkonzepte, Funktionsweise und Bestandteile** von Adversarial Autoencoder Netzen zu verstehen.
>2. Eine **Vorverarbeitung** von kategorischen Finanzdaten (d.h. One-Hot Encoding und Min-Max Normalisierung) durchzuführen. 
>3. Adversarial Autoencoder Netze anzuwenden, um **Anomalien** in umfangreichen Finanzdaten aufzuspüren.
>4. Die **Ergebnisse** bzw. den Latenten Raum von Adversarial Autoencoder Netzen zu interpretieren.

## 1. Einrichten der Analyseumgebung

Ähnlich wie in den vorangegangenen Übungen werden wir zunächst eine Reihe von Python-Bibliotheken importieren, welche die Datenanalyse und -visualisierung ermöglichen. In dieser Übung werden wir die Bibliotheken `PyTorch`, `Pandas`, `Numpy`, `Scikit-Learn`, `Matplotlib` und `Seaborn` verwenden. Nachfolgend importieren wir die benötigten Bibliotheken durch die Ausführung der folgenden Anweisungen:

In [ ]:
# import python data science and utility libraries
import os, sys, itertools, urllib, io, warnings
import datetime as dt
import pandas as pd
import pandas_datareader as dr
import numpy as np
import random as rd

Import der `PyTorch` Deep Learning Bibliotheken:

In [ ]:
# pytorch libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torch.utils.data import dataloader

Import der `Matplotlib` und `Seaborn` Visualisierungs Bibliotheken und setzen der Visualisierungsparameter:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi']= 150

Ausschalten möglicher Warnmeldungen z.B. aufgrund von zukünftigen Änderungen der Bibliotheken:

In [ ]:
# set the warning filter flag to ignore warnings
warnings.filterwarnings('ignore')

Aktivieren der sog. Inline-Darstellung von Visualisierungen in Jupyter-Notebook:

In [ ]:
%matplotlib inline

Erstellen von Unterverzeichnissen innerhalb des aktuellen Arbeitsverzeichnisses für (1) die `GDrive` Notebooks im Allgemeinen, (2) das Speichern der Originaldaten, (3) der Analyseergebnisse und (4) der trainierten Modelle:

In [ ]:
# create Colab Notebooks directory
notebook_directory = '/content/drive/MyDrive/Colab Notebooks'
if not os.path.exists(notebook_directory): os.makedirs(notebook_directory)

 # create data sub-directory inside the Colab Notebooks directory
data_directory = '/content/drive/MyDrive/Colab Notebooks/01_data'
if not os.path.exists(data_directory): os.makedirs(data_directory)

# create results sub-directory inside the Colab Notebooks directory
results_directory = '/content/drive/MyDrive/Colab Notebooks/02_results'
if not os.path.exists(results_directory): os.makedirs(results_directory)

 # create models sub-directory inside the Colab Notebooks directory
models_directory = '/content/drive/MyDrive/Colab Notebooks/03_models'
if not os.path.exists(models_directory): os.makedirs(models_directory)

Festlegen eines zufälligen Seeds zur Gewährleistung der Reproduzierbarkeit:

In [ ]:
# init deterministic seed
seed_value = 1234
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value); # set pytorch seed cpu

Aktivieren des GPU Computing, durch setzen des `device` flag und setzen eines zufälligen `CUDA` Seeds:

In [ ]:
# set cpu or gpu enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type

# set pytorch gpu seed
torch.cuda.manual_seed(seed_value)

# log type of device enabled
now = dt.datetime.utcnow().strftime("%Y.%m.%d-%H:%M:%S")
print('[LOG {}] notebook with {} computation enabled'.format(str(now), str(device)))

Anzeige der Hardware Informationen zu den ggf. verfügbaren GPU(s):

In [ ]:
!nvidia-smi

Anzeige der Software Informationen über die verfügbaren `Python` bzw. `PyTorch` Versionen:

In [ ]:
# print current Python version
now = dt.datetime.utcnow().strftime("%Y.%m.%d-%H:%M:%S")
print('[LOG {}] The Python version: {}'.format(now, sys.version))

In [ ]:
# print current PyTorch version
now = dt.datetime.utcnow().strftime("%Y.%m.%d-%H:%M:%S")
print('[LOG {}] The PyTorch version: {}'.format(now, torch.__version__))

## 2. Datenakquise und Datenaufbereitung

Heutzutage beschleunigen Unternehmen die Digitalisierung von Geschäftsprozessen, wovon auch Enterprise Resource Planning (ERP)-Systeme betroffen sind. Diese Systeme sammeln grosse Mengen Daten auf granularer Ebene. Dies gilt insbesondere für die Journalbuchungen einer Organisation, die innerhalb des Hauptbuch und den jeweiligen Nebenbüchern erfasst werden.

Die Darstellung in **Abbildung 1** zeigt eine hierarchische Ansicht eines ERP-Systems, das Journalbuchungen in Datenbanktabellen erfasst. Im Kontext revisorischer Prüfungen können die in solchen Systemen erfassten Daten Spuren bzw. wertvolle Hinweise auf mögliche dolose Handlungen enthalten.

<img align="middle" style="max-width: 600px; height: auto" src="./accounting.png">

**Abbildung 1:** Hierarchische Ansicht eines Enterprise Resource Planning (ERP)-Systems, das Geschäftsvorfälle auf verschiedene Abstraktionsebenen in Datenbanktabellen erfasst, d.h. auf Ebene (1) des Geschäftsprozesses, (2) der Buchhaltung sowie (3) der Datenbank.

Zunächst werden wir den im Rahmen des Labs verwendeten Datensatzes deskriptiv analysieren. Anschliessend werden wir die Daten vorverarbeiten um eine Ausgangslage für das Training eines Neuronalen Netzes zu schaffen. Der Lab Datensatz basiert auf einer angepassten Teilmenge des **"Synthetic Financial Dataset For Fraud Detection "** Datensatz von Lopez-Rojas. Der Originaldatensatz wurde ursprünglich über die Kaggle-Plattform für Data Science Wettbewerbe veröffentlicht und kann über den nachfolgenden Link abgerufen werden kann: https://www.kaggle.com/ntnu-testimon/paysim1.

In einem ersten Schritt laden wir den Datensatz in unsere Analyseumgebung:

In [ ]:
# load the dataset into the notebook
url = 'https://raw.githubusercontent.com/GitiHubi/CADS/main/lab_06/fraud_dataset_v2.csv'
ori_dataset = pd.read_csv(url)

Anschliessend prüfen wir die Dimensionalität des Datensatzes:

In [ ]:
# inspect the datasets dimensionalities
now = dt.datetime.utcnow().strftime("%Y.%m.%d-%H:%M:%S")
print('[LOG {}] transactional dataset of {} rows and {} columns retreived.'.format(now, ori_dataset.shape[0], ori_dataset.shape[1]))

Darüber hinaus speichern wir eine Sicherheitskopie des geladenen Datensatzes mit aktuellem Zeitstempel:

In [ ]:
# determine current timestamp 
timestamp = dt.datetime.utcnow().strftime("%Y-%m-%d_%H-%M-%S")

# define dataset filename 
filename = timestamp + " - original_fraud_dataset.xlsx"

# save dataset extract to the data directory
ori_dataset.head(1000).to_excel(os.path.join(data_directory, filename))

### 2.1 Initiales Daten Assessment

Der Datensatz enthält insgesamt **sieben kategorische** und **zwei numerische Attribute**, welche den innerhalb eines SAP FICO Moduls enthaltenen Tabellen BKPF (Buchungsbelegköpfe) und BSEG (Buchungsbelegsegmente) entsprechen. Die nachstehende Liste enthält einen Überblick über die einzelnen Attribute sowie eine kurze Beschreibung ihrer jeweiligen Semantik:

>- `BELNR`: die Nummer des Buchhaltungsbelegs,
>- `BUKRS`: der Buchungskreis
>- `BSCHL`: der Buchungsschlüssel,
>- `HKONT`: das gebuchte Hauptbuchkonto,
>- `PRCTR`: das gebuchte Profit Center,
>- `WAERS`: der Währungsschlüssel,
>- `KTOSL`: der Schlüssel des Hauptbuchkontos,
>- `DMBTR`: der Betrag in der Hauswährung,
>- `WRBTR`: der Betrag in der Belegwährung.

Sehen wir uns auch einmal die ersten 10 Zeilen des Datensatzes im Detail an:

In [ ]:
# inspect top rows of dataset
ori_dataset.head(10) 

Vielleicht ist Ihnen bei der Durchsicht der Attribute auch das Attribut mit der Bezeichung `Label` in den Daten aufgefallen. Dieses Attribut enthält die **Ground-Truth Informationen** zu den jeweils einzelnen Buchungen. Das Attribut beschreibt die 'wahre Natur' jeder Transaktion, d.h. ob es sich um eine **reguläre** Transaktion (gekennzeichnet durch `regulär`) oder eine **Anomalie** (gekennzeichnet durch `global` und `lokal`) handelt.  

Innerhalb unseres Vorgehens werden wir die Label Information nur dazu verwenden, um die Ergebnisse unserer trainierten Modelle zu validieren. Bitte beachten Sie jedoch, dass uns eine solches Feld in der Realität oftmals nicht zur Verfügung steht. Schauen wir uns nun einmal die Verteilung der regulären gegenüber den anomalen Buchungen im Datensatz an:

In [ ]:
# number of anomalies vs. regular transactions
ori_dataset.label.value_counts()

Die Analyse zeigt, dass wir es, ähnlich wie in der realen Welt, mit einem **unbalanzierten Datensatz** konfrontiert sind. D.h. insgesamg enthält der Datensatz nur einen sehr kleinen Anteil von **100 (0,109 %)** anomalen Transaktionen. Unter den 100 Anomalien befinden sich **70 (0,076 %)** *globale* Anomalien und **30 (0,003 %)** *lokale* Anomalien. 

In einem nächsten Schritt entfernen wir das `label` Attribut aus dem Trainingsdatensatz und speichern es in einer gesonderten Variable:

In [ ]:
# remove the "ground-truth" label information for the following steps of the class
label = ori_dataset.pop('label')

### 2.2 Vorverarbeitung der Kategorischen Attribute

Aus der Sichtung der Daten geht hervor, dass die Mehrzahl der Attribute kategorische (diskrete) Attributwerte aufweisen, z.B. das Buchungsdatum, das Hauptbuchkonto, die Buchungsart und die Währung. Schauen wir uns nun die Verteilung der kategorischen Attribute *Buchungsschlüssel* `BSCHL` sowie *Hauptbuchkonto* `HKONT` einmal im Detail an:

In [ ]:
# prepare to plot posting key and general ledger account side by side
fig, ax = plt.subplots(1, 2)
fig.set_figwidth(18)

# plot the distribution of the posting key attribute
plot = sns.countplot(x=ori_dataset['BSCHL'], ax=ax[0])

# set axis labels
plot.set_xticklabels(plot.get_xticklabels(), rotation=90)
plot.set_xlabel('Attribute Value', fontsize=16)
plot.set_ylabel('Attribute Value Count', fontsize=16)

# set plot title
plot.set_title('Buchungsschlüssel Attribute Value Distribution', fontsize=16)

# plot the distribution of the general ledger attribute
plot = sns.countplot(x=ori_dataset['HKONT'], ax=ax[1])

# set axis labels
plot.set_xticklabels(plot.get_xticklabels(), rotation=90)
plot.set_xlabel('Attribute Value', fontsize=16)
plot.set_ylabel('Attribute Value Count', fontsize=16)

# set plot title
plot.set_title('Hauptbuchkonto Attribute Value Distribution', fontsize=16);

Im Allgemeinen sind Neuronale Netze dafür konzipiert numerische Daten zu verarbeiten. Eine Möglichkeit, diese Anforderung zu erfüllen, ist die Anwendung eines Verfahrens, das als sog. **One-Hot Kodierung** bezeichnet wird. Hierdurch kann eine numerische Darstellung kategorischer Attributwerte abgeleitet werden. Bei der **One-Hot Kodierung** wird für jeden kategorischen Attributwert eine zusätzliche binäre Spalte in den Daten erstellt. 

Schauen wir uns hierzu das Beispiel in **Abbildung 2** unten an. Das kategorische Attribut **Receiver** in den Orginaldaten enthält die Namen 'Sally', 'John' und 'Emma'. Wir kodieren das Attribut als 'one-hot' Attribut, indem wir eine zusätzliche binäre Spalte für jeden kategorischen Wert in der Spalte 'Receiver' erstellen. Anschliessend kodieren wir z.B. jede Transaktion, die den Wert 'Sally' in der Spalte 'Receiver' aufweist mit dem Wert 1.0 innerhalb der 'Sally' Spalte der Transaktion. Sollte eine Transaktion einen anderen Wert in der Spalte 'Receiver' aufweisen, kodieren wir die 'Sally' Spalte mit dem Wert 0.0. 

<img align="middle" style="max-width: 500px; height: auto" src="./encoding.png">

**Abbildung 2:** Beispielhafte 'One-Hot' Kodierung der verschiedenen Receiver Attributwerte in spezifische binäre 'One-Hot' Spalten. Dabei resultiert jeder im Datensatz beobachtbare Attributwert in einer eigene Spalte. Der Spaltenwert **1.0** kodiert das Vorkommen des Attributwertes in der entsprechenden Buchung. Der Spaltenwert **0.0** hingegen zeigt, dass der Attributwert nicht innerhalb der entsprechenden Buchung vorkommt.

Anhand dieses Verfahrens können die insgesamt sechs kategorischen Attribute des Datensatzes in numerische Attribute überführt werden. Die `Pandas` Bibliothek stellt hierzu die entsprechende Funktionalität zur Verfügung, welche wir im Nachfolgenden anwenden:

In [ ]:
# select categorical attributes to be "one-hot" encoded
categorical_attr_names = ['KTOSL', 'PRCTR', 'BSCHL', 'HKONT', 'BUKRS', 'WAERS']

# encode categorical attributes into a binary one-hot encoded representation 
ori_dataset_cat_processed = pd.get_dummies(ori_dataset[categorical_attr_names])

Nachfolgend überprüfen wie die vorgenommene **One-Hot Kodierung** anhand der 10 ersten Buchungen des Datensatzes:

In [ ]:
# inspect encoded sample transactions
ori_dataset_cat_processed.head(10)

### 2.3 Vorverarbeitung der numerischen Attribute

Anschliessend Analysieren wir nun die Verteilungen der **beiden numerischen Attribute** des Datensatzes. Hierbei handelt es sich um die Attribute (1) *Betrag in Hauswährung* `DMBTR` und (2) *Betrag in Dokumentwährung* `WRBTR` deren jeweilge Verteilungen wir nachfolgend visualisieren: 

In [ ]:
# plot the log-scaled 'DMBTR' as well as the 'WRBTR' attribute value distribution
fig, ax = plt.subplots(1,2)
fig.set_figwidth(18)

# plot distribution of the local amount attribute
plot = sns.distplot(ori_dataset['DMBTR'].tolist(), ax=ax[0])

# set axis labels
plot.set_xlabel('Attribute Value', fontsize=16)
plot.set_ylabel('Attribute Value Count', fontsize=16)

# set plot title
plot.set_title('Betrag in Hauswährung - Attribute Value Distribution', fontsize=16)

# plot distribution of the document amount attribute
plot = sns.distplot(ori_dataset['WRBTR'].tolist(), ax=ax[1])

# set axis labels
plot.set_xlabel('Attribute Value', fontsize=16)
plot.set_ylabel('Attribute Value Count', fontsize=16)

# set plot title
plot.set_title('Betrag in Dokumentwährung - Attribute Value Distribution', fontsize=16);

Die Werte beider Betragsattribute weisen eine jeweils **schiefe** und **steile** Verteilung auf. Wir skalieren deshalb die Werte zunächst logarithmisch. Anschliessend min-max normalisieren wir die Skalierten Werte:

In [ ]:
# select the 'DMBTR' and 'WRBTR' attribute
numeric_attr_names = ['DMBTR', 'WRBTR']

# add a small epsilon to eliminate zero values from data for log scaling
numeric_attr = ori_dataset[numeric_attr_names] + 1e-7

# log scale the 'DMBTR' and 'WRBTR' attribute values
numeric_attr = numeric_attr.apply(np.log)

# normalize all numeric attributes to the range [0,1]
ori_dataset_num_processed = (numeric_attr - numeric_attr.min()) / (numeric_attr.max() - numeric_attr.min())

In einem nächsten Schritt visualisieren wir die Verteilungen der skalierten bzw. normierten Werte beider Betragsattribute:

In [ ]:
# plot the log-scaled 'DMBTR' as well as the 'WRBTR' attribute value distribution
fig, ax = plt.subplots(1,2)
fig.set_figwidth(18)

# plot distribution of the local amount attribute
plot = sns.distplot(ori_dataset_num_processed['DMBTR'].tolist(), ax=ax[0])

# set axis labels
plot.set_xlabel('Attribute Value', fontsize=16)
plot.set_ylabel('Attribute Value Count', fontsize=16)

# set plot title
plot.set_title('Betrag in Hauswährung - Attribute Value Distribution', fontsize=16)

# plot distribution of the document amount attribute
plot = sns.distplot(ori_dataset_num_processed['WRBTR'].tolist(), ax=ax[1])

# set axis labels
plot.set_xlabel('Attribute Value', fontsize=16)
plot.set_ylabel('Attribute Value Count', fontsize=16)

# set plot title
plot.set_title('Betrag in Dokumentwährung - Attribute Value Distribution', fontsize=16);

### 2.4 Merge Categorical and Numerical Transaction Attributes

Abschliessend fügen wir die beiden vorverarbeiteten numerischen und kategorischen Attribute zu einem **einzigen Datensatz** zusammen. Der zusammengeführte Datensatz bildet die Grundlage für das nachfolgende Training des Deep Autoencoder Neural Networks (AENNs):

In [ ]:
# merge categorical and numeric subsets
ori_subset_transformed = pd.concat([ori_dataset_cat_processed, ori_dataset_num_processed], axis = 1)

Werfen wir nun abschliessend noch einen finalen einen Blick auf die Dimensionalität des zusammengefügten Datensatzes:

In [ ]:
# inspect final dimensions of pre-processed transactional data
ori_subset_transformed.shape

Nach Abschluss der Vorverarbeitungsschritte verfügen wir über einen Datensatz, der aus einer Gesamtzahl von **533,009 Datensätzen** (Zeilen) und **618 Attributen** (Spalten) besteht. Wir behalten die Anzahl der Spalten im Hinterkopf, da sie die Dimensionalität der Eingabe- und Ausgabeschicht unseres AENNs bestimmen wird.

## 3. Adversarial Autoencoder Neural Network Implementierung

In diesem Abschnitt möchten wir uns mit der zugrundeliegenden Idee und dem Aufbau eines **Adversarial Autoencoder Neural Networks (AAENNs)** vertraut zu machen. Hierzu werden wir die einzelne Bausteine und die spezifische Netzwerkstruktur von AAENNs anhand der `PyTorch` Open-Source-Bibliothek implementieren.

### 3.1 Adversarial Autoencoder Neural Network Architektur

Die Architektur des **Adversarial Autoencoder Neural Networks (AAENNs)**, wie in der Abbildung unten dargestellt, erweitert das Konzept klassischer Autoencoder Neural Networks (AENNs), um die Idee sog. **Generative Adversarial Networks (GANs)**. Anhand dieser Erweitung ist es dem AAENN möglich die erlernten Repräsentationen innerhalb des latenten Raum des AAENNs anhand einer beliebigen Prior-Verteilung azuordnen. Das Lernen selbst erfolgt pro Trainingsbatch in zwei Phasen, (1) einer Rekonstruktionsphase und (2) einer Regularisierungsphase.

Innerhalb der **Rekonstruktionsphase** wird der AAENN-Encoder $q_{\theta}(z|x)$ trainiert, um eine aggregierte Posterior-Verteilung $q(z)$ der Journaleinträge $X$ über den latenten Raum $Z$ zu lernen. Dabei entspricht die gelernte Posterior-Verteilung, in Analogie zu AENNs, komprimierten Repräsentatonen der Journalbuchungen. Das  Decoder-Netzwerk $p_{\theta}(\hat{x}|z)$ des AAENNs verwendet die erlernten Repräsentationen $z_i \in Z$ wiederum, um die Journaleinträge so Originalgetreu wie möglich zu rekonstruieren $\hat{X}$. **Zielsetzung der Rekonstruktionsphase ist es den Rekonstruktionsfehler der AAENNs zu minimieren**.

In der **Regularisierungsphase** wird ein adversarial Trainingsverfahren angewandt bei dem das Encoder-Netzwerk $q_{\theta}(z|x)$ zugleich als Generator-Netzwerk fungiert. Zusätzlich erfolgt der Aufsatz eines Diskriminator-Netzwerk $d_{\theta}(z)$ auf den latenten Raum $Z$ des AAENNs. In Analogie zum Training von GANs wird das Diskriminator-Netzwerk des AAENN trainiert, um die Samples einer ausgewählten Prior-Verteilung $p(z) \in Z$, sog. **Fake Verteilung**, von der gelernten Posterior-Verteilung $q(z)$, sog. **True Verteilung**, zu unterscheiden. **Zielsetzung der Regulierungsphase ist es das Diskriminator-Netzwerk des AAENNs maximal zu täuschen**. 

Eine solche Täuschung gelingt dem Generator-Netzwerk wenn sich Fake und True Verteilung $p(z) ≈ q(z)$ nicht mehr signifikant von einander unterscheiden. Das heisst, sich das Diskriminator-Netzwerk nicht sicher ist, ob ein vorliegendes Trainingssample aus der **True Verteilung** $q(z)$ oder der **Fake Verteilung** $p(z)$ stammt.

<img align="middle" style="max-width: 830px; height: auto" src="https://github.com/GitiHubi/deepAD/blob/master/images/autoencoder_2.png?raw=1">

**Abbildung 3:** Schematische Darstellung eines **Adversarial Autoencoder Neural Networks (AAENNs)**, das aus drei nicht-linearen Abbildungen bzw. Feed-Forward-Netzen besteht. Die drei miteinander verknüpften Netze werden als Encoder-Netzwerk $q_\theta: \mathbb{R}^{dx} \mapsto \mathbb{R}^{dz}$, Decoder-Netzwerk $p_\theta: \mathbb{R}^{dz} \mapsto \mathbb{R}^{dx}$, und Discriminator-Netzwerk $d_\phi: \mathbb{R}^{dz} \mapsto \mathbb{R}^{d1}$ bezeichnet.

In [ ]:
#Todo

### 3.1 Encoder Netzwerk Implementierung

In einem nächsten Schritt möchten wir nun das **Encoder Netz $q_{\theta}(z | x)$** in `PyTorch` implementieren. Der Encoder soll aus insgesamt **fünf Schichten von fully-connected Neuronen** bestehen. Darüber hinaus solle der Encoder die nachfolgende Anzahl von Neuronen pro Schicht enthalten: 618-256-64-16-4-2. Die vorhergehende Notation bedeutet, dass die erste Schicht 618 Neuronen umfasst (bestimmt durch die Dimensionalität der Eingabedaten), die zweite Schicht 256 Neuronen und die weiteren Schichten 64, 16, 4 bzw. 2 Neuronen.

Den nachfolgenden drei Elementen der Implementierung des Encoder Netzes möchten wir eine besonderes Augenmerk schenken:

>- `self.encoder_Lx`: definiert die lineare Transformation der jeweiligen Schicht, welche auf die Eingabe angewandt wird: $Wx + b$.
>- `nn.init.xavier_uniform`: initialisiert Gewichtsparameter anhand einer gleichmäßigen Xavier Verteilung.
>- `nn.init.constant`: initialisiert den Bias pro Schicht mit einem konstanten Wert 0.0.
>- `self.encoder_Rx`: definiert die nicht-lineare Transformation der jeweiligen Schicht, welche auf die Eingabe angewandt wird: $\sigma(\cdot)$.

Wir verwenden sog. **Leaky ReLUs**, um saturierende bzw. 'sterbende' Neuronen zu vermeiden und die Trainingskonvergenz zu beschleunigen. Die Anwendung von Leaky ReLUs ermöglicht die Berechnung von Gradienten auch innerhalb des negativen Bereichs einer Aktivierungsfunktion (siehe vorhergehendes Lab 05).

In [ ]:
# implementation of the encoder network
class encoder(nn.Module):

    # define class constructor
    def __init__(self):

        # call super class constructor
        super(encoder, self).__init__()

        # specify layer 1 - in 618, out 256
        self.map_L1 = nn.Linear(in_features=ori_subset_transformed.shape[1], out_features=256, bias=True) # init linearity
        nn.init.xavier_uniform_(self.map_L1.weight) # init weights according to [9]
        nn.init.constant_(self.map_L1.bias, 0.0) # constant initialization of the bias
        self.map_R1 = nn.LeakyReLU(negative_slope=0.4, inplace=True) # add non-linearity according to [10]

        # specify layer 2 - in 256, out 64
        self.map_L2 = nn.Linear(256, 64, bias=True)
        nn.init.xavier_uniform_(self.map_L2.weight)
        nn.init.constant_(self.map_L2.bias, 0.0)
        self.map_R2 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 3 - in 64, out 16
        self.map_L3 = nn.Linear(64, 16, bias=True)
        nn.init.xavier_uniform_(self.map_L3.weight)
        nn.init.constant_(self.map_L3.bias, 0.0)
        self.map_R3 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 4 - in 16, out 4
        self.map_L4 = nn.Linear(16, 4, bias=True)
        nn.init.xavier_uniform_(self.map_L4.weight)
        nn.init.constant_(self.map_L4.bias, 0.0)
        self.map_R4 = torch.nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 5 - in 4, out 2
        self.map_L5 = nn.Linear(4, 2, bias=True)
        nn.init.xavier_uniform_(self.map_L5.weight)
        nn.init.constant_(self.map_L5.bias, 0.0)
        self.map_R5 = torch.nn.LeakyReLU(negative_slope=0.4, inplace=True)
        
    # define forward pass
    def forward(self, x):

        # run forward pass through the network
        x = self.map_R1(self.map_L1(x))
        x = self.map_R2(self.map_L2(x))
        x = self.map_R3(self.map_L3(x))
        x = self.map_R4(self.map_L4(x))
        x = self.map_R5(self.map_L5(x))

        # return result
        return x

In einem nächsten Schritt instanzieren wir ein Modell des Encoder Netzes:

In [ ]:
# intstantiate the encoder network model
encoder_train = encoder()

Anschliessend transferieren wir das Encoder Modell auf die `CPU` oder eine ggf. verfügbare `GPU`:

In [ ]:
# push model to compute device
encoder_train = encoder_train.to(device)

Sofern verfügbar, prüfen wir ob das Modell erfolgreich auf die `GPU`  übertragen wurde:

In [ ]:
!nvidia-smi

Nun können wir die Modellstruktur visualisieren und die Netzarchitektur nochmals durch das Ausführen der folgenden Zelle überprüfen:

In [ ]:
# print the initialized architectures
now = dt.datetime.utcnow().strftime("%Y.%m.%d-%H:%M:%S")
print('[LOG {}] encoder architecture:\n\n{}\n'.format(now, encoder_train))

### 3.2 Decoder Netzwerk Implementierung

In einem nächsten Schritt vervollständigen wir nun die Autoencoder Architektur des AAENNs durch die Implementierung des entsprechenden **Decoder Netzes $p_{\theta}(\hat{x}|z)$**. Der Decoder soll ebenfall aus insgesamt **fünf Schichten** von fully-connected Neuronen bestehen. Zudem soll der Decoder die Architektur des Encoders  **symmetrisch spiegeln**. Wir invertieren hierzu die Ausgestaltung der Schichten des Encoders schichtweise, gemäss der folgenden Struktur 2-4-16-64-256, im Rahmen der Implementierung des Decoders:

In [ ]:
# implementation of the decoder network
class decoder(nn.Module):

    # define class constructor
    def __init__(self):

        # call super class constructor
        super(decoder, self).__init__()

        # specify layer 1 - in 2, out 4
        self.map_L1 = nn.Linear(in_features=2, out_features=4, bias=True) # init linearity
        nn.init.xavier_uniform_(self.map_L1.weight) # init weights according to [9]
        nn.init.constant_(self.map_L1.bias, 0.0) # constant initialization of the bias
        self.map_R1 = nn.LeakyReLU(negative_slope=0.4, inplace=True) # add non-linearity according to [10]

        # specify layer 2 - in 4, out 16
        self.map_L2 = nn.Linear(4, 16, bias=True)
        nn.init.xavier_uniform_(self.map_L2.weight)
        nn.init.constant_(self.map_L2.bias, 0.0)
        self.map_R2 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 3 - in 16, out 64
        self.map_L3 = nn.Linear(16, 64, bias=True)
        nn.init.xavier_uniform_(self.map_L3.weight)
        nn.init.constant_(self.map_L3.bias, 0.0)
        self.map_R3 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 4 - in 64, out 256
        self.map_L4 = nn.Linear(64, 256, bias=True)
        nn.init.xavier_uniform_(self.map_L4.weight)
        nn.init.constant_(self.map_L4.bias, 0.0)
        self.map_R4 = nn.LeakyReLU(negative_slope=0.4, inplace=True)
        
        # specify layer 5 - in 256, out 618
        self.map_L5 = nn.Linear(256, ori_subset_transformed.shape[1], bias=True)
        nn.init.xavier_uniform_(self.map_L5.weight)
        nn.init.constant_(self.map_L5.bias, 0.0)
        self.map_S5 = torch.nn.Sigmoid()
        
    # define forward pass
    def forward(self, x):

        # run forward pass through the network
        x = self.map_R1(self.map_L1(x))
        x = self.map_R2(self.map_L2(x))
        x = self.map_R3(self.map_L3(x))
        x = self.map_R4(self.map_L4(x))
        x = self.map_S5(self.map_L5(x))

        # return result
        return x

Wir instanzieren nun auch das Decoder Modell für das `CPU` bzw. `GPU`Training und überzeugen uns davon, dass das Modell erfolgreich initialisiert wurde. Hierzu visualisieren wir wieder die Netzarchitektur durch das Ausführen der nachfolgenden Zelle:

In [ ]:
# intstantiate the decoder network model
decoder_train = decoder()

# push model to compute device
decoder_train = decoder_train.to(device)
    
# print the initialized architectures
now = dt.datetime.utcnow().strftime("%Y.%m.%d-%H:%M:%S")
print('[LOG {}] decoder architecture:\n\n{}\n'.format(now, decoder_train))

### 3.2 Discriminator Netzwerk Implementierung

In einem finalen Schritt erweitern wir nun die Autoencoder Architektur des AAENNs um das dritte notwendige Netzwerk. Dies geschieht durch die Implementierung des **Discriminator Netzes $d_{\theta}(z)$**. Der Discriminator soll aus insgesamt **vier Schichten** von fully-connected Neuronen bestehen. Hierbei ist zu beachten, dass die **Eingabeschicht des Discriminators** die gleiche Dimensionalität wie die **Ausgabeschicht des Encoders** aufweist. Die vier Schichten des Discriminators umfassen die nachfolgende Anzahl Neuronen pro Schicht 256-16-4-2.

In [ ]:
# implementation of the discriminator network
class discriminator(nn.Module):

    # define class constructor
    def __init__(self):

        # call super class constructor
        super(discriminator, self).__init__()

        # specify first layer - in 2, out 256
        self.discriminator_L1 = nn.Linear(2, 256, bias=True) # init linearity
        nn.init.xavier_uniform_(self.discriminator_L1.weight) # init weights according to [9]
        nn.init.constant_(self.discriminator_L1.bias, 0.0) # constant initialization of the bias
        self.discriminator_R1 = nn.LeakyReLU(negative_slope=0.4, inplace=True) # add non-linearity according to [10]

        # specify second layer - in 256, out 16
        self.discriminator_L2 = nn.Linear(256, 16, bias=True)
        nn.init.xavier_uniform_(self.discriminator_L2.weight)
        nn.init.constant_(self.discriminator_L2.bias, 0.0)
        self.discriminator_R2 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify third layer - in 16, out 4
        self.discriminator_L3 = nn.Linear(16, 4, bias=True)
        nn.init.xavier_uniform_(self.discriminator_L3.weight)
        nn.init.constant_(self.discriminator_L3.bias, 0.0)
        self.discriminator_R3 = nn.LeakyReLU(negative_slope=0.4, inplace=True)
        
        # specify fourth layer - in 4, out 2
        self.discriminator_L4 = nn.Linear(4, 2, bias=True)
        nn.init.xavier_uniform_(self.discriminator_L4.weight)
        nn.init.constant_(self.discriminator_L4.bias, 0.0)
        self.discriminator_S4 = torch.nn.Sigmoid()

    # define forward pass
    def forward(self, x):

        # run forward pass through the network
        x = self.discriminator_R1(self.discriminator_L1(x))
        x = self.discriminator_R2(self.discriminator_L2(x))
        x = self.discriminator_R3(self.discriminator_L3(x))
        x = self.discriminator_S4(self.discriminator_L4(x))

        # return result
        return x

Wir instanzieren nun auch das Discriminator Modell für das `CPU` bzw. `GPU`Training und überzeugen uns davon, dass das Modell erfolgreich initialisiert wurde. Hierzu visualisieren wir wieder die Netzarchitektur durch das Ausführen der nachfolgenden Zelle:

In [ ]:
# intstantiate the discriminator network model
discriminator_train = discriminator()

# push model to compute device
discriminator_train = discriminator_train.to(device)
    
# print the initialized architectures
now = dt.datetime.utcnow().strftime("%Y.%m.%d-%H:%M:%S")
print('[LOG {}] discriminator architecture:\n\n{}\n'.format(now, discriminator_train))

Abschliessend werfen wir noch einen Blick auf die Anzahl der Modellparameter, die wir im Folgenden beabsichtigen zu trainieren:

In [ ]:
# init the number of encoder model parameters
encoder_num_params = 0

# iterate over the distinct encoder parameters
for param in encoder_train.parameters():

    # collect number of parameters
    encoder_num_params += param.numel()

# init the number of decoder model parameters
decoder_num_params = 0
    
# iterate over the distinct decoder parameters
for param in decoder_train.parameters():

    # collect number of parameters
    decoder_num_params += param.numel()

# init the number of discriminator model parameters
discriminator_num_params = 0
    
# iterate over the distinct discriminator parameters
for param in discriminator_train.parameters():

    # collect number of parameters
    discriminator_num_params += param.numel()
    
# print the number of model paramters
now = dt.datetime.utcnow().strftime("%Y.%m.%d-%H:%M:%S")
print('[LOG {}] number of to be trained AAENN model parameters: {}.'.format(now, encoder_num_params + decoder_num_params + discriminator_num_params))

Ok, unser AAENN Modell umfasst eine beachtliche Gesamtzahl von **357.634 zu trainierenden Modellparametern**.

### 3.4 Adversarial Autoencoder Neural Network Training

Nachdem wir nun das AAENN implementiert haben, möchten wir in einem nächsten Schritt das Netz trainieren. Bevor wir jedoch mit dem Training beginnen, ist es notwendig geeignete Fehlerfunktionen zu definieren. Zur Erinnerung: Wir wollen das Modell jeweils in zwei Trainingsphasen trainieren, nämlich (1) einer **Rekonstruktionsphase** und (2) einer **Regularisierungsphase**. Im Folgenden werden wir die Trainingsparameter für jede der Beiden Trainingsphasen festlegen.

#### 3.4.1 Parametrisierung der Rekonstruktionsphase

In der Rekonstruktionsphase wird der Encoder $q_{\theta}(z|x)$ trainiert, um eine aggregierte Posterior-Verteilung $q(z)$ der Journalbuchungen $X$ innerhalb des latenten Raums $Z$ zu lernen. Dabei entspricht die gelernte Posterior-Verteilung komprimierten Repräsentationen der Merkmale der Journalbuchungen. In Analogie zu klassischen Autoencoder Neuronalen Netzen nutzt der Decoder $p_{\theta}(\hat{x}|z)$ die gelernten latenten Repräsentationen $z_{i} \in Z$, um die Journaleinträge so getreu wie möglich zu rekonstruieren $\hat{X}$ und den Rekonstruktionsfehler der AAENNs zu minimieren.

Um dieses Lernziel zu erreichen, verwenden wir im folgenden unterschiedliche Rekonstruktionsfehler entsprechend der Art der zu rekonstruierenden Buchungsmerkmale. Für kategorische Merkmalsausprägungen $x^{i}_{cat}$, z.B. verwenden wir den **Binary-Cross-Entropy (BCE)** Rekonstruktionsfehler. Für numerische Merkmalsausprägungen $x^{i}_{con}$, z.B. , verwenden wir den **Mean-Squared-Error (MSE)** Rekonstruktionsfehler. Abschliessend werden pro zu rekonstruierender Journalbuchung $x^{i}$ beide Fehler additiv, wie nachfolgend, miteinander verknüpft:

<center> $\mathcal{L}_{\theta}^{REC}(x;\hat{x}) = \gamma \hspace{1mm} \mathcal{L}^{BCE}_{\theta}(x_{cat};\hat{x}_{cat}) + (1 - \gamma) \hspace{1mm} \mathcal{L}^{MSE}_{\theta}(x_{con};\hat{x}_{con})$ </center>

wobei $x^{i}$, $i=1,...,n$ die Menge an Buchungen bezeichnet, $\hat{x}^{i}$ die jeweiligen Rekonstruktionen und $j=1,...,k$ die verschiedenen Buchungsattribute indexiert. Nachfolgend instanzieren wir die entsprechende BCE Fehlerfunktion $\mathcal{L}^{BCE}$ und MSE Fehlerfunktion $\mathcal{L}^{MSE}$ der `PyTorch` Bibliothek:

In [ ]:
# define the bce optimization criterion / loss function
reconstruction_loss_categorical = nn.BCELoss(reduction='mean')

# define the mse optimization criterion / loss function
reconstruction_loss_numerical = nn.MSELoss(reduction='mean')

Anschliessend transferieren wir die Berechnung der beiden Fehlerfunktionen auf die `CPU` oder eine ggf. verfügbare `GPU`:

In [ ]:
# push the optimization criterion / loss function to compute device 
reconstruction_loss_categorical = reconstruction_loss_categorical.to(device)
reconstruction_loss_numerical = reconstruction_loss_numerical.to(device)

Auf der Grundlage der Fehlerhöhe eines Mini-Batches an Buchungen berechnet die `PyTorch` Bibliothek automatisiert die Gradienten. Anschliessend werden AAENN-Parameter $\theta$ auf Grundlage der ermittelten Gradienten optimiert. Hierzu ist es notwendig das gewünschte Optimierungsverfahren in **PyTorch** zu definieren. In der nachfolgenden Notebook Zelle verwenden wir das sog. **Adam Optimierungsverfahren** für die Optimierung der Modellparameter $\theta$. Darüber hinaus definieren eine Lernrate $l = 0.001$ sowohl für die Optimierung des Encoder als auch das Decoder Netzwerk:

In [ ]:
# set encoder and decoder learning rate
learning_rate_encoder = 1e-3
learning_rate_decoder = 1e-3

# define encoder and decoder optimization strategy
encoder_optimizer = optim.Adam(encoder_train.parameters(), lr=learning_rate_encoder)
decoder_optimizer = optim.Adam(decoder_train.parameters(), lr=learning_rate_decoder)

#### 3.4.2 Parametrisierung der Regulierungsphase

In der Regularisierungsphase wird ein adversarial Trainingsverfahren angewandt, in welchem der Encoder $q_{\theta}(z|x)$ des AAENN als Generator fungiert. Zusätzlich wird ein Diskriminator-Netzwerk $d_{\phi}(z)$ auf den gelernten latenten Raum $Z$ aufgesetzt. In Analogie zum Training von Generative Adversarial Networks (GANs) wird der Discriminator des AAENN trainiert. Die Zielsetzung des Trainining ist es hierbei Samples einer **Fake Prior-Verteilung $p(z)$** in $z_{i} \in Z$ von der erlernten **True Posterior-Verteilung $q(z)$** zu unterscheiden. Prallel hierzu wird der Encoder mit der Zielsetzung trainiert, eine **True Posterior-Verteilung** der Art zu lernen, dass $p(z) \approx q(z)$. Hierdurch wird dem Diskriminator maximal verwirrt vorgaukelt, dass die aus $q(z)$ gezogenen Samples aus $p(z)$ stammen und umgekehrt. Eine hierfür geeignete Fehlerfunktion findet sich im sog. **Binary-Cross-Entropy (BCE)** Klassifikationsfehler, der formal wie nachfolgend definiert ist:

<center> $\mathcal{L^{Dis}_{\phi}}(y^{i};\hat{y}^{i}) = \frac{1}{n}\sum_{i=1}^{n} y^{i} ln(\hat{y}^{i}) + (1-y^{i}) ln(1-\hat{y}^{i})$, </center>

wobei $y^{i}$ das tatsächliche Klassenlabel der Verteilung (Fake Prior oder True Posterior Verteilung) bezeichnet und $\hat{y}^{i}$ das durch den Discriminator vorhergesagte Klassenlabel.  Nachfolgend instanzieren wir die entsprechende BCE Fehlerfunktion der `PyTorch` Bibliothek:

In [ ]:
# define the bce optimization criterion / loss function
discriminator_loss = nn.BCELoss()

Anschliessend transferieren wir die Berechnung der Fehlerfunktion auf die `CPU` oder eine ggf. verfügbare `GPU`:

In [ ]:
# push the optimization criterion / loss function to compute device 
discriminator_loss = discriminator_loss.to(device)

Auf der Grundlage der BCE Fehlerhöhe eines Mini-Batches von latenten Samples berechnet die `PyTorch` Bibliothek automatisiert die Gradienten. Anschliessend werden die Parameter des Discriminators $\phi$ auf Grundlage der ermittelten Gradienten optimiert. In der nachfolgenden Notebook Zelle verwenden wir das sog. **Adam Optimierungsverfahren** für die Optimierung der $\phi$ Parameter. Darüber hinaus definieren eine Lernrate $l = 0.00001$:

In [ ]:
# set generator and discriminator learning rate
learning_rate_discriminator = 1e-5

# set generator and discriminator optimization strategy
discriminator_optimizer = optim.Adam(discriminator_train.parameters(), lr=learning_rate_discriminator)

Nachdem wir die drei Netze des aAENN-Modells erfolgreich implementiert und instanziiert haben. Nehmen wir uns Zeit, die Definition der Modelle **(1) Encoder**, **(2) Decoder** und **(3) Discriminator** sowie die Fehlerfunktionen nochmals zu überprüfen und etwaige Fragen zu besprechen.

### 3.5 Gaussian Mixture Prior Implementierung

In einem nächsten Schritt erstellen wir eine Prior-Verteilung, welches es uns ermöglicht im Laufe des Trainingsprozesses zu samplen. Um die durch den AAENN gelernten Repräsentationen der Journalbuchungen im latenten Raum $Z$ zu partitionieren samplen wir aus einer **Prior-Verteilung $p(z)$, die aus einem Mixture von $\tau$ multivariaten isotropen Gauss-Verteilungen $\mathcal{N}(\mu,\mathcal{I})$ besteht**. Um die Ergebnisse im Nachfolgenden visualisieren zu können samplen wir aus einer zwei dimensionalen Prior-Verteilung $\mu \in \mathcal{R}^{2}$ ist. Nachfolgend möchten wir die die Journalbuchungen in insgesamt fünf generative Prozesse partitionieren. Hierzu erstellen wir eine Prior-Verteilung $p(z)$, die insgesamt $\tau=5$ isotropen Gauss-Verteilungen umfasst. 

Hierzu definieren wir zunächst die nachfolgenden vier Parameter: Anzahl Gauss-Verteilungen $\tau$, Radius der einzelnen Verteilungen $r$, Standardabweichungen der Verteilungen $\sigma$ und die Dimension der Verteilungen $\mathcal{R}^{d}$:

In [ ]:
# define the number of gaussians
tau = 5 

# define radius of each gaussian
radius = 0.8

# define the sigma of each gaussian
sigma = 0.01

# define the dimensionality of each gaussian
dim = 2

In einem nächsten Schritt, erstellen wir die gewünschte **Prior-Verteilung $p(z)$**. Hierzu samplen wir jeweils 1,000 zufällige samples von jeder Gauss-Verteilung der gewünschten Prior-Verteilung:

In [ ]:
# determine x and y coordinates of the target mixture of gaussians
x_centroid = (radius * np.sin(np.linspace(0, 2 * np.pi, tau, endpoint=False)) + 1) / 2
y_centroid = (radius * np.cos(np.linspace(0, 2 * np.pi, tau, endpoint=False)) + 1) / 2

# determine each gaussians mean (centroid) and standard deviation
mu_gauss = np.vstack([x_centroid, y_centroid]).T

# determine the number of samples to be created per gaussian
samples_per_gaussian = 100000

# iterate over the number of distinct gaussians
for i, mu in enumerate(mu_gauss):

    # case: first gaussian distribution
    if i == 0:

        # randomly sample from gaussian distribution 
        z_continous_samples_all = np.random.normal(mu, sigma, size=(samples_per_gaussian, dim))

    # case: non-first gaussian distribution
    else:

        # randomly sample from gaussian distribution
        z_continous_samples = np.random.normal(mu, sigma, size=(samples_per_gaussian, dim))

        # collect and stack new samples
        z_continous_samples_all = np.vstack([z_continous_samples_all, z_continous_samples])

Abschliessend möchten wir nun die gewünschte Prior-Verteilung $p(z)$ visuell prüfen:

In [ ]:
# init the plot
fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111)

# plot prior distribution scatter plot
ax.scatter(z_continous_samples_all[:, 0], z_continous_samples_all[:, 1], c='C0', marker="o", edgecolors='w', linewidth=0.5) 

# add axis labels
ax.set_xlabel('$z_1$')
ax.set_ylabel('$z_2$')

# add plot title
ax.set_title('Prior Distribution $p(z)$', size=10);

Ok, die Samples der gewünschten Prior-Verteilung $p(z)$ entsprechen der erwarteten Verteilung. Anschliessend werden wir die Verteilung nun im Rahmen des AAENN Trainings verwenden.

## 4. Adversarial Autoencoder Neural Network Network

In diesem Abschnitt möchten wir nun ein AAENN-Modell anhand der kodierten Transaktionsdaten trainieren. Darüber hinaus werfen wir einen detaillierten Blick auf die einzelnen Trainingshyperparameter und Trainingsschritte sowie den Trainingsfortschritt im Zeitverlauf. 

### 4.1 Definition der Hyperparameter

Beginnen wir nun damit, ein AAENN Modell für **5 Trainingsepochen** und **128 Buchungen pro Mini-Batch** zu trainieren. Diese Konfiguration der Hyperparameter bedeutet, dass der gesamte Datensatz dem AAENN ingesamt fünf Mal in Mini-Batches von 128 jeweils Buchungen zugeführt wird. Diese Hyperparameter Konfiguration hat zu Folge, dass pro Trainingsepoche **4,165 Updates** (533,009 Buchungen modulo 128 Buchungen pro Mini-Batch) der AAENN Modellparameter erfolgen. 

In [ ]:
# specify training parameters
num_epochs = 5 # number of training epochs
mini_batch_size = 128 # size of the mini-batches

Während der Trainingsphase sollen dem AENN Modell kontinuierlich Mini-Batches der gesamten Population von Buchungen zugeführt werden. Hierzu verwenden wir die `DataLoader` Funktionalität der `PyTorch` Bibliothek. Dabei handelt es sich im sog. Iteratoren, welche die Buchungen kontinuierlich in Form von Mini-Batches zur Verfügung stellen. Durch die Aktivierung des Parameters `shuffle=True` werden die Daten in jeder Epoche neu gemischt, bevor sie dem Netz zugeführt werden. In der nachfolgenden Zelle instanzieren wir einen PyTorch Dataloader der Buchungsdaten:

In [ ]:
# convert pre-processed transactional data to PyTorch tensor
torch_dataset = torch.from_numpy(ori_subset_transformed.values).float()

# push pre-processed transactional data to compute device
torch_dataset = torch_dataset.to(device)

# init the dataloader
dataloader_train = dataloader.DataLoader(torch_dataset, batch_size=mini_batch_size, shuffle=True)

### 4.2 Training des Models

In einenm nächsten Schritt starten wir nun das Modelltraining. Für jeden zugeführten Mini-Batch erfolgt der Trainingsprozess in zwei Phasen, der **(1) Rekonstruktionsphase** und der **(2) Regularisierungsphase**. Innerhalb der Rekonstruktionsphase durchläuft das Training die nachfolgenden Schritte: 

>1. Durchführung des Forward-pass durch das Encoder- und Decoder-Net.
>2. Berechnen des Rekonstruktionsfehlers $\mathcal{L^{Rec}_{\theta}}(x^{i};\hat{x}^{i})$.
>3. Durchführung des Backwardpass durch das Decoder- und Encoder-Netz.
>4. Update der Encoder $q_\theta(\cdot)$- und Decoder $p_\theta(\cdot)$ Parameter.

Innerhalb der nachfolgende Regulierungsphase wird zunächst das Discriminator-Netz anhand der nachfolgenden Schritte trainiert: 

>1. Durchführung des forward pass durch das Generator-Netz (bzw. Encoder-Netz) um die **fake** Repräsentation $z^{i}_{fake} \sim q_\theta(\cdot)$ zu erhalten.
>2. Sampling einer **true** Repräsentation aus der gewünschten Prior-Verteilung $z^{i}_{real} \sim p(\cdot)$.
>3. Berechnen des Diskriminationsfehlers $\mathcal{L}_{\theta}^{Dis}(z^{i}_{fake}; z^{i}_{real})$.
>4. Update der Discriminator-Netz $d_\phi(\cdot)$ Parameter. 

Anschliessend erfolgt, innerhalb der Regularisierungsphase, das Training des Generator-Netzes (bzw. Encoder Netzes) anhand der nachfolgenden Schritte: 

>1. Durchführung des forward pass durch das Generator-Netz (bzw. Encoder-Netz) um die **fake** Repräsentation $z^{i}_{fake} \sim q_\theta(\cdot)$ zu erhalten.
>2. Berechnen des Diskriminationsfehlers $\mathcal{L}_{\theta}^{Dis}(z^{i}_{fake}; z^{i}_{real})$.
>3. Update der Generator-Netz (bzw. Encoder-Netz) $q_\theta(\cdot)$ Parameter.

Um das Lernen während des Trainings AAENN-Modells zu gewährleisten, werden wir anschliessend beobachten, ob die unterschiedlichen Netzwerkfehler mit fortschreitendem Training abnehmen. Pro durchgeführter Trainingsepoche ermitteln wir deshalb kontinuierlich den Rekonstruktionsfehler $\mathcal{L}_{\theta}^{Rec}$, den Diskriminationsfehler $\mathcal{L}_{\theta}^{DIS}$ und den Generationsfehler $\mathcal{L}_{\theta}^{GEN}$. Die Analyse der Fehler ermöglicht es uns zu evaluieren, ob die verschiedenen Netze lernen bzw. die Fehler konvergieren.

Nach jeder abgeschlossenen Trainingsepoche möchten wir zudem einen sog. **Modell Checkpoint** pro Netz speichern. Die Checkpoints enthalten eine Bestandsaufnahme bzw. 'Schnappschuss' der Modellparameter. Im Allgemeinen ist es eine gute Praxis, während des Trainings solche Checkpoints in regelmässigen Abständen zu speichen. Sollte das Training einmal unterbrochen werden, kann es beginnend auf dem letzten Checkpoint wieder fortgesetzt werden. Für das Speichern eines Modell Checkpoints verwenden wir die nachfolgende `PyTorch` Anweisung:

>- `torch.save()`: speichert den Checkpoint der aktuellen Modellparameterwerte auf dem lokalen Dateisystem.

Anschliessend möchten das Training eines AAENN Modells beginnen:

In [ ]:
# init collection of training losses
epoch_reconstruction_losses = []
epoch_discriminator_losses = []
epoch_generator_losses = []

# initialize training adversarial autoencoder model
for epoch in range(num_epochs):

    # init mini batch counter
    mini_batch_count = 0
    
    # init epoch training losses
    batch_reconstruction_losses = 0.0
    batch_discriminator_losses = 0.0
    batch_generator_losses = 0.0

    # set networks in training mode
    encoder_train.train()
    decoder_train.train()
    discriminator_train.train()
    
    # start timer
    start_time = dt.datetime.now()

    # iterate over epoch mini batches
    for mini_batch_data in dataloader_train:

        # increase mini batch counter
        mini_batch_count += 1
  
        # push mini batch data to compute device
        mini_batch_data = mini_batch_data.to(device)
        
        # reset the networks gradients
        encoder_train.zero_grad()
        decoder_train.zero_grad()
        discriminator_train.zero_grad()

        # =================== reconstruction phase =====================
        
        # run autoencoder encoding - decoding
        z_sample = encoder_train(mini_batch_data)
        mini_batch_reconstruction = decoder_train(z_sample)

        # split input mini-batch data into numerical and categorical part
        batch_cat = mini_batch_data[:, :ori_dataset_cat_processed.shape[1]]
        batch_num = mini_batch_data[:, ori_dataset_num_processed.shape[1]:]
        
        # split reconstructed mini-batch data intoi numerical and categorical part
        rec_batch_cat = mini_batch_reconstruction[:, :ori_dataset_cat_processed.shape[1]]
        rec_batch_num = mini_batch_reconstruction[:, ori_dataset_num_processed.shape[1]:]

        # determine categorical and numerical reconstruction loss
        rec_error_categorical = reconstruction_loss_categorical(input=rec_batch_cat, target=batch_cat)  # one-hot attr error
        rec_error_numerical = reconstruction_loss_numerical(input=rec_batch_num, target=batch_num)  # numeric attr error

        # determine combined reconstruction loss
        reconstruction_loss_all = rec_error_categorical + rec_error_numerical
        
        # run backward pass - determine gradients
        reconstruction_loss_all.backward()
        
        # collect batch reconstruction loss
        batch_reconstruction_losses += reconstruction_loss_all.item()
        
        # update network parameter - decoder and encoder
        decoder_optimizer.step()
        encoder_optimizer.step()

        # =================== regularization phase =====================
        # =================== discriminator training ===================

        # set discriminator in evaluation mode
        discriminator_train.eval()

        # generate true latent space data
        z_true_batch = z_continous_samples_all[rd.sample(range(0, z_continous_samples_all.shape[0]), mini_batch_size),:]

        # convert to torch tensor and push to compute device
        z_true_batch = torch.FloatTensor(z_true_batch).to(device)

        # determine mini batch sample generated by the encoder -> fake gaussian sample
        z_fake_gauss = encoder_train(mini_batch_data)

        # determine discriminator classification of both samples
        d_true_gauss = discriminator_train(z_true_batch) # true sampled gaussian 
        d_fake_gauss = discriminator_train(z_fake_gauss) # fake created gaussian

        # create discriminator label of true and fake sample 
        d_true_gauss_target = torch.ones(d_true_gauss.shape).to(device) # true -> 1
        d_fake_gauss_target = torch.zeros(d_fake_gauss.shape).to(device) # fake -> 0

        # determine individual discrimination losses
        discriminator_loss_true = discriminator_loss(target=d_true_gauss_target, input=d_true_gauss) # true loss
        discriminator_loss_fake = discriminator_loss(target=d_fake_gauss_target, input=d_fake_gauss) # fake loss
        
        # combine true loss and fake loss
        discriminator_loss_all = discriminator_loss_fake + discriminator_loss_true

        # run backward through the discriminator network
        discriminator_loss_all.backward()
        
        # collect discriminator loss
        batch_discriminator_losses += discriminator_loss_all.item()

        # update network the discriminator network parameters
        discriminator_optimizer.step()

        # reset the networks gradients
        encoder_train.zero_grad()
        decoder_train.zero_grad()
        discriminator_train.zero_grad()

        # =================== regularization phase =====================
        # =================== generator training =======================

        # set encoder / generator in training mode
        encoder_train.train()
        
        # reset the encoder / generator networks gradients
        encoder_train.zero_grad()

        # determine fake gaussian sample generated by the encoder / generator
        z_fake_gauss = encoder_train(mini_batch_data)

        # determine discriminator classification of fake gaussian sample
        d_fake_gauss = discriminator_train(z_fake_gauss)

        # determine discriminator classification target variables
        d_fake_gauss_target = torch.FloatTensor(torch.ones(d_fake_gauss.shape)).to(device) # fake -> 1

        # determine discrimination loss of fake gaussian sample
        generator_loss = discriminator_loss(target=d_fake_gauss_target, input=d_fake_gauss)
        
        # collect generator loss
        batch_generator_losses += generator_loss.item()

        # run backward pass - determine gradients
        generator_loss.backward()

        # update network paramaters - encoder / generatorc
        encoder_optimizer.step()

        # reset the networks gradients
        encoder_train.zero_grad()
        decoder_train.zero_grad()
        discriminator_train.zero_grad()

    # collect epoch training losses - reconstruction loss
    epoch_reconstruction_loss = batch_reconstruction_losses / mini_batch_count
    epoch_reconstruction_losses.extend([epoch_reconstruction_loss])
    
    # collect epoch training losses - discriminator loss
    epoch_discriminator_loss = batch_discriminator_losses / mini_batch_count
    epoch_discriminator_losses.extend([epoch_discriminator_loss])
    
    # collect epoch training losses - generator loss
    epoch_generator_loss = batch_generator_losses / mini_batch_count
    epoch_generator_losses.extend([epoch_generator_loss])
    
    # print epoch reconstruction loss
    now = dt.datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
    print('[LOG TRAIN {}] epoch: {:04}/{:04}, reconstruction loss: {:.4f}'.format(now, epoch + 1, num_epochs, epoch_reconstruction_loss))
    print('[LOG TRAIN {}] epoch: {:04}/{:04}, discriminator loss: {:.4f}'.format(now, epoch + 1, num_epochs, epoch_discriminator_loss))
    print('[LOG TRAIN {}] epoch: {:04}/{:04}, generator loss: {:.4f}'.format(now, epoch + 1, num_epochs, epoch_generator_loss))
    
    # =================== save model snapshots to disk ============================
    
    # save trained encoder model file to disk
    now = dt.datetime.utcnow().strftime("%Y%m%d-%H_%M_%S")
    encoder_model_name = "{}_ep_{}_encoder_model.pth".format(now, (epoch+1))
    torch.save(encoder_train.state_dict(), os.path.join(models_directory, encoder_model_name))

    # save trained decoder model file to disk
    decoder_model_name = "{}_ep_{}_decoder_model.pth".format(now, (epoch+1))
    torch.save(decoder_train.state_dict(), os.path.join(models_directory, decoder_model_name))
    
    # save trained discriminator model file to disk
    decoder_model_name = "{}_ep_{}_discriminator_model.pth".format(now, (epoch+1))
    torch.save(discriminator_train.state_dict(), os.path.join(models_directory, decoder_model_name))

In einem nächsten Schritt visualisieren wir die jeweiligen Rekonstruktionsfehler bzw. Diskriminationsfehler für jede Trainingsepoche:

In [ ]:
# init the plot
fig = plt.figure(figsize=(4, 2))
ax = fig.add_subplot(111)

# plot the reconstruction loss per training epoch
plt.plot(range(1, len(epoch_reconstruction_losses)+1), epoch_reconstruction_losses)

# set plot axis labels
plt.xlabel('Training Epoch', size=10)
plt.ylabel('Reconstruction Loss', size=10)

# set plot title
plt.title('AAENN Model Training', size=10);

In [ ]:
# init the plot
fig = plt.figure(figsize=(4, 2))
ax = fig.add_subplot(111)

# plot the discriminator loss per training epoch
plt.plot(range(0, len(epoch_discriminator_losses)), epoch_discriminator_losses)

# set plot axis labels
plt.xlabel('Training Epoch', size=10)
plt.ylabel('Discrimination Loss', size=10)

# set plot title
plt.title('AAENN Model Training', size=10);

In [ ]:
# init the plot
fig = plt.figure(figsize=(4, 2))
ax = fig.add_subplot(111)

# plot the generator loss per training epoch
plt.plot(range(0, len(epoch_generator_losses)), epoch_generator_losses)

# set plot axis labels
plt.xlabel('Training Epoch', size=10)
plt.ylabel('Generation Loss', size=10)

# set plot title
plt.title('AAENN Model Training', size=10);

Nach den ersten fünf Trainingsepochen ist zu beobachten, dass der **Rekonstruktionsfehler kontinuierlich abnihmmt**. Dies deutet darauf hin, dass es dem Adversarial Autoencoder zunehment gelingt die Journalbuchungen korrekt zu rekonstruieren.

Aus den Schaubildern wird auch deutlich, dass das Modelltraining noch nicht konvergiert. D.h. das AAENN Modell könnte noch einige weitere Epochen trainiert werden. Um im Rahmen des Labs jedoch etwas Zeit zu sparen, möchten wir das Notebook nun mit einem bereits für 400 Epochen vortrainierten Modell fortsetzen.

## 5. Evaluating the Autoencoder Neural Network (AENN) Model

In diesem Abschnitt möchten wir die Fähigkeit des erlernten AAENN Modells zur Erkennung von Anomalien in Buchhaltungsdaten evaluieren. Hierzu werden wir auf vortrainierte AENN Modelle zurück greifen. Die Evaluation umfasst die **lokalen** als auch die **globalen** Anomalien des Datensatzes.

### 5.1 Laden der Modell Checkpoints

Für die Evaluation laden wir üblicherweise das AAENN-Modell mit **geringstem Rekonstruktions- bzw. Diskriminationsfehler**. Pro Trainingsepoche wurde im Rahmen des Modelltrainings jeweils ein Checkpoint der Modellparameter innerhalb des lokalen Modellverzeichnis gespeichert. Wir werden nun die bereits für **400 Trainingsepochen** trainierten Modell Checkpoint laden:

In [ ]:
# restore pretrained model checkpoint
encoder_model_name = 'https://raw.githubusercontent.com/GitiHubi/CADS/main/lab_06/03_models/20220818-03_22_18_ep_401_encoder_model.pth'
decoder_model_name = 'https://raw.githubusercontent.com/GitiHubi/CADS/main/lab_06/03_models/20220818-03_22_18_ep_401_decoder_model.pth'

# Read stored model from the remote location
encoder_bytes = urllib.request.urlopen(encoder_model_name)
decoder_bytes = urllib.request.urlopen(decoder_model_name)

# Load tensor from io.BytesIO object
encoder_buffer = io.BytesIO(encoder_bytes.read())
decoder_buffer = io.BytesIO(decoder_bytes.read())

# init training network classes / architectures
encoder_eval = encoder().to(device)
decoder_eval = decoder().to(device)
    
# load trained models
encoder_eval.load_state_dict(torch.load(encoder_buffer, map_location=lambda storage, loc: storage))
decoder_eval.load_state_dict(torch.load(decoder_buffer, map_location=lambda storage, loc: storage))

### 5.2 Visualisierung des Latenten Raums

In einem nächsten Schritt, möchten wir nun den latenten Raum $Z$ inkl. der erlernten Repräsentationen $z_{i}$ des bereits vortrainierten Modells visualisieren. Hierzu definieren wir uns zunächst einen Dataloader, welcher die Journalbuchungen dem AAENN-Modell zur Verfügung stellt ohne die Buchungen zu mischen:

In [ ]:
# convert pre-processed data to pytorch tensor
torch_dataset = torch.from_numpy(ori_subset_transformed.values).float()

# convert to pytorch tensor - none cuda enabled
dataloader_eval = dataloader.DataLoader(torch_dataset, batch_size=mini_batch_size, shuffle=False)

Anschliessend berechnen wir die erlernte Repräsentation $z_{i}$ für jede Journalbuchung $z_{i}$ des Datensatzes. Hierzu führen wir einen forward pass durch das Encoder-Netz für jede Journalbuchung durch:

In [ ]:
# set networks in evaluation mode
encoder_eval.eval()
decoder_eval.eval()

# init batch count
batch_count = 0

# iterate over epoch mini batches
for enc_transactions_batch in dataloader_eval:

    # determine latent space representation of all transactions
    z_enc_transactions_batch = encoder_eval(enc_transactions_batch)
    
    # case: initial batch 
    if batch_count == 0:

      # collect reconstruction errors of batch
      z_enc_transactions_all = z_enc_transactions_batch
      
    # case: non-initial batch
    else:
      
      # collect reconstruction errors of batch
      z_enc_transactions_all = torch.cat((z_enc_transactions_all, z_enc_transactions_batch), dim=0)
    
    # increase batch count
    batch_count += 1

# convert to numpy array
z_enc_transactions_all = z_enc_transactions_all.cpu().detach().numpy()

In einem Folgeschritt, möchten wir die erlernte Repräsentation $z_{i}$ jeder Journalbuchung $x_{i}$ innerhalb des latenten Raums $Z \in \mathcal{R}^{2}$ visualisieren. Visualiserung soll zudem kennzeichnen, ob es sich bei einer jeweiligen Journalbuchung um eine reguläre Buchung oder eine Anomalie handelt:

In [ ]:
# prepare plot
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111)

# obtain regular transactions as well as global and local anomalies
regular_data = z_enc_transactions_all[label == 'regular']
global_outliers = z_enc_transactions_all[label == 'global']
local_outliers = z_enc_transactions_all[label == 'local']

# plot reconstruction error scatter plot
ax.scatter(regular_data[:, 0], regular_data[:, 1], c='C0', marker="o", label='regular', edgecolors='w', linewidth=0.5) # plot regular transactions
ax.scatter(global_outliers[:, 0], global_outliers[:, 1], c='C1', marker="x", label='global', edgecolors='w', s=60) # plot global outliers
ax.scatter(local_outliers[:, 0], local_outliers[:, 1], c='C3', marker="x", label='local', edgecolors='w', s=60) # plot local outliers

# add plot legend of transaction classes
ax.legend(loc='best');

### 5.3 Berechnung der Normalisierten Mean-Divergence 

Grundsätzlich lässt sich beobachten, dass Journalbuchungen die ungewöhnliche Attributwerte aufweisen (globale Anomalien, siehe Lab 05), in Regionen mit geringer Wahrscheinlichkeitsdichte des latenten Raum ihre Anordnung finden. D.h. im Falle der Prior-Verteilung dieses Notebooks, eine erhöhte Divergenz von den Means des Mixture Gaussians $\mathcal{N}(\mu,\mathcal{I})$ aufweisen. Der Parameter $\tau$ bezeichnet die Anzahl der der verschiedenen Gaussians, die durch $\mu =\{\mu^1 \ldots \mu^\tau \}, t=1...\tau$ definiert sind und $\mu^{t} \in \mathcal{R}^m$. 

Anhand der Regulierungsphase des AAENN-Trainings werden die Repräsentationen der Journalbuchungen grds. in Richtung der Regionen mit hoher Wahrscheinlichkeitsdichte der Prior-Verteilung 'geschoben'. Zugleich werden durch die Rekonstruktionsphase, Repräsentationen die seltenen oder anomalen Journalbuchungen entsprechen in Regionen mit geringer Wahrscheinlichkeitsdichte platziert. Dies hat zur Folge, dass reguläre Journalbuchungen eine geringere Distanz zu den Means der Prior-Verteilung aufweisen. 

Nachfolgend ermitteln wir für jede Repräsentation $z^{i}$ den euklidischen Abstand zu ihrem nächstgelegenen Mean $\mu^\tau$. Formal kann die Mean-Divergenz kann anhand der nachfolgenden Formel berechnet werden $D_{\theta^*}^{\tau}(z^{i};\mu) = \min\limits_{\tau} \lVert z^i-\mu^\tau \rVert^2$. Für die Ermittlung eines Anomaly-Scores globaler Anomalien berechnen wir die normalisierte Mean-Divergenz $MD$ unter gegebenen (optimalen) Modellparametern $\theta^*$, ausgedrückt durch:

\begin{equation}
MD_{\theta^*}^{\tau}(x^{i}) = \frac{D_{\theta^*}^{\tau}(z^i;\mu) - D_{\theta^*, min}^{\tau}}{D_{\theta^*, max}^{\tau} - D_{\theta^*, min}^{\tau}},
\end{equation}

wobei $D_{min}$ und $D_{max}$ die Minimal- und Maximalwerte des erhaltenen Mean-Divergenz bezeichnen, welche durch $\mathcal{L}_{\theta^*}$ und den jeweils nächsten Modus $\tau$ der Buchung determiniert werden. Für die Berechnung der Mean-Divergenz implementieren wir zunächst die **Euklidische Distanz** zwischen zwei Punkten:

In [ ]:
# define euclidean distance calculation
def compute_euclid_distance(x, y):
    
    # calculate euclidean distance 
    euclidean_distance = np.sqrt(np.sum((x - y) ** 2, axis=1))
    
    # return euclidean distance
    return euclidean_distance

Anschliessend berechnen wir die Mean-Divergence $D_{\theta}^{\tau}$ der Repräsentationen $z_{i}$ der Journalbuchungen:

In [ ]:
# determine distance to each mode
distances = np.apply_along_axis(func1d=compute_euclid_distance, axis=1, arr=z_enc_transactions_all, y=mu_gauss)

# determine mode divergence
mean_divergence = np.min(distances, axis=1)

# determine min-mode id
cluster_ids = np.argmin(distances, axis=1)    

In einem nächsten Schritt, normalisieren wir die erhaltene Mean-Divergence pro Journalbuchung $D_{\theta^*}^{\tau}$ anhand der die Minimal- und Maximalwerte $D_{min}$ und $D_{max}$ des jeweils zugehörigen Means $\tau$ der Prior-Verteilung:

In [ ]:
# prepare empty arrays of the same shape and dtype
mean_divergence_all_scaled = np.asarray(mean_divergence)

# iterate over the cluster means
for cluster_id in np.unique(cluster_ids).tolist():
  
    # determine journal entries of current mode
    mask = cluster_ids == cluster_id

    # normalize journal entries mean divergence to the range [0,1]
    mean_divergence_all_scaled[mask] = (mean_divergence[mask] - mean_divergence[mask].min()) / (mean_divergence[mask].ptp())   

Abschliessend visualisieren wir die normalisierte Mean-Divergence $MD_{\theta^*}^{\tau}$ der Repräsentationen $z^{i}$ der Journalbuchungen $x^{i}$:

In [ ]:
# collect anomaly score, labels and cluster assignments
plot_data = pd.concat([pd.Series(mean_divergence_all_scaled, name='mean_divergence'), 
                       pd.Series(label, name='label'),                        
                       pd.Series(cluster_ids, name='cluster_id')],
                     axis=1)

# determine number of clusters
num_clusters = len(np.unique(cluster_ids))

# init sub-plots based on the number of modes
fig, axes = plt.subplots(1, num_clusters, sharey=True, figsize=(14, 10))

# iterate over distinct modes
for mode in range(0, num_clusters):

    # sample data fraction for faster visualization
    plot_data = plot_data.sample(frac=1.0)
    
    # collect features of current mode
    z_mode = plot_data[plot_data['cluster_id'] == mode]
    
    # seperate regular and anomalous journal entries
    regular_data = z_mode[z_mode['label'] == 'regular']
    global_outliers = z_mode[z_mode['label'] == 'global']
    local_outliers = z_mode[z_mode['label'] == 'local']

    # create train scatter plot of regular samples
    axes[mode].scatter(regular_data.index, regular_data['mean_divergence'], c='C0', marker='o', s=30, linewidth=0.3, label='regular', edgecolors='w')
    
    # create train scatter plot of global anomalies
    axes[mode].scatter(global_outliers.index, global_outliers['mean_divergence'], c='C1', marker='x', s=120, linewidth=3, label='global', edgecolors='w')
    
    # create train scatter plot of local anomalies
    axes[mode].scatter(local_outliers.index, local_outliers['mean_divergence'], c='C3', marker='x', s=120, linewidth=3, label='local', edgecolors='w')

    # set axis labels
    xlabel = '$\\tau={}$' + str(mode+1) if mode == 0 else str(mode+1)
    axes[mode].set_xlabel(xlabel, fontsize=20)

    # set y-axis limits
    axes[mode].set_ylim([0.0, 1.1])

    # set x-axis ticks and labels
    axes[mode].set_xticks([int(plot_data.shape[0]/2)])
    axes[mode].set_xticklabels(['$x_{i}$'])

# set axis labels
axes[0].set_ylabel('Mean-Divergence $MD$', fontsize=20)

# init plot legend
handles, labels = axes[2].get_legend_handles_labels()

# plot legend
plt.legend(handles, labels, loc='center', fontsize=20, ncol=3, borderaxespad=0., bbox_to_anchor=(-6.5, 1., 9., .1))

# set grid plot layout
plt.grid(True)

### 5.4 Berechnung des Normalisierten Rekonstruktionsfehlers

Grundsätzlich lässt sich beobachten, dass Journalbuchungen die ungewöhnliche Korrelationen der Attributwerte aufweisen (lokale Anomalien, siehe Lab 05), in einem hohen Rekonstruktionsfehler resultieren. Dies resultiert aus der Rekonstruktionsphase des AAENN-Architektur. Durch die verlustbehaftete Kompression des Encoders werden ungewöhnliche bzw. Anomale Attributskorrelationen nicht durch das AAENN Model gelernt. Dies resultiert zudem in einer Überlagerung von regulären Buchungen und lokalen Anomalien im latenten Raum. 

Nachfolgend ermitteln wir für jede Journalbuchung $x^{i}$ den kombinierten Rekonstruktionsfehler $\mathcal{L}^{Rec}$ bestehend aus MSE und BCE Fehler (siehe Definition oben). Für jede Journalbuchung wird anschliessend der normalisierte Rekonstruktionsfehler $\mathcal{L}^{Rec}$ unter gegebenen (optimalen) Modellparametern $\theta^*$ berechnet, ausgedrückt durch:

\begin{equation}
RE_{\theta^*}^{\tau}(x^{i};\hat{x}^{i}) = \frac{\mathcal{L}_{\theta^*}^{Rec, \tau}(x^i;\hat{x}^{i}) - \mathcal{L}_{\theta^*, min}^{Rec, \tau}}{\mathcal{L}_{\theta^*, max}^{Rec, \tau} - \mathcal{L}_{\theta^*, min}^{Rec, \tau}},
\end{equation}

wobei $\mathcal{L}_{min}^{Rec}$ und $\mathcal{L}_{max}^{Rec}$ die Minimal- und Maximalwerte der erhaltenen Rekonstruktionsfehler bezeichnen, welche durch $\mathcal{L}_{\theta^*}$ und den jeweils nächsten Modus $\tau$ der Buchung determiniert werden. Nachfolgend ermitteln wir zunächst den Rekonstruktionsfehler $\mathcal{L}_{\theta^*}^{Rec, \tau}$ für jeden Journaleintrag $x^{i}$:

In [ ]:
# define the optimization criterion / loss function
reconstruction_criterion_categorical_eval = nn.BCEWithLogitsLoss(reduction='none').to(device)
reconstruction_criterion_numeric_eval = nn.MSELoss(reduction='none').to(device)

# set networks in evaluation mode
encoder_eval.eval()
decoder_eval.eval()

# init batch count
batch_count = 0

# iterate over epoch mini batches
for enc_transactions_batch in dataloader_eval:

    # determine latent space representation of all transactions
    z_enc_transactions_batch = encoder_eval(enc_transactions_batch)

    # reconstruct input samples
    reconstruction_batch = decoder_eval(z_enc_transactions_batch)

    # split input transactions into numeric and categorical parts
    input_cat_all = enc_transactions_batch[:, :ori_dataset_cat_processed.shape[1]]
    input_num_all = enc_transactions_batch[:, ori_dataset_num_processed.shape[1]:]

    # split reconstruction into numeric and categorical parts
    rec_cat_all = reconstruction_batch[:, :ori_dataset_cat_processed.shape[1]]
    rec_num_all = reconstruction_batch[:, ori_dataset_num_processed.shape[1]:]

    # compute rec error
    rec_error_cat_all = reconstruction_criterion_categorical_eval(input=rec_cat_all, target=input_cat_all).mean(dim=1)
    rec_error_num_all = reconstruction_criterion_numeric_eval(input=rec_num_all, target=input_num_all).mean(dim=1)

    # combine categorical and numerical errors
    rec_error_all_batch = rec_error_cat_all + rec_error_num_all
    
    # case: initial batch
    if batch_count == 0:
    
      # collect reconstruction errors of batch
      rec_error_all = rec_error_all_batch
    
    # case: non-initial batch
    else:
      
      # collect reconstruction errors of batch
      rec_error_all = torch.cat((rec_error_all, rec_error_all_batch), dim=0)
    
    # increase batch count
    batch_count += 1

# convert to numpy array
rec_error_all = rec_error_all.cpu().detach().numpy()

In einem nächsten Schritt, normalisieren wir den erhaltenen Rekonstruktionsfehler pro Journalbuchungen $\mathcal{L}^{Rec}$ anhand der Minimal- bzw. Maximalwerte der erhaltenen Rekonstruktionsfehler $\mathcal{L}_{min}^{Rec}$ bzw. $\mathcal{L}_{max}^{Rec}$ pro zugehörigem Mean $\tau$ der Prior-Verteilung:

In [ ]:
# prepare empty arrays of the same shape and dtype
rec_error_all_scaled = np.asarray(rec_error_all)

# iterate over the cluster modes
for cluster_id in np.unique(cluster_ids).tolist():
  
    # determine journal entries of current mode
    mask = cluster_ids == cluster_id

    # normalize mode journal entries reconstruction error to the range [0,1]
    rec_error_all_scaled[mask] = (rec_error_all[mask] - rec_error_all[mask].min()) / (rec_error_all[mask].ptp())

Abschliessend visualisieren wir den normalisierten Rekonstruktionsfehler $RE_{\theta^*}^{\tau}$ der Journalbuchungen $x^{i}$:

In [ ]:
# collect anomaly score, labels and cluster assignments
plot_data = pd.concat([pd.Series(rec_error_all_scaled, name='rec_error'), 
                       pd.Series(label, name='label'),                        
                       pd.Series(cluster_ids, name='cluster_id')],
                     axis=1)

# determine number of clusters
num_clusters = len(np.unique(cluster_ids))

# init sub-plots based on the number of modes
fig, axes = plt.subplots(1, num_clusters, sharey=True, figsize=(14, 10))

# iterate over distinct modes
for mode in range(0, num_clusters):

    # sample data fraction for faster visualization
    plot_data = plot_data.sample(frac=1.0)
    
    # collect features of current mode
    z_mode = plot_data[plot_data['cluster_id'] == mode]

    # seperate regular and anomalous journal entries
    regular_data = z_mode[z_mode['label'] == 'regular']
    global_outliers = z_mode[z_mode['label'] == 'global']
    local_outliers = z_mode[z_mode['label'] == 'local']

    # create train scatter plot of regular samples
    axes[mode].scatter(regular_data.index, regular_data['rec_error'], c='C0', marker='o', s=30, linewidth=0.3, label='regular', edgecolors='w')
    
    # create train scatter plot of global anomalies
    axes[mode].scatter(global_outliers.index, global_outliers['rec_error'], c='C1', marker='x', s=120, linewidth=3, label='global', edgecolors='w')
    
    # create train scatter plot of local anomalies
    axes[mode].scatter(local_outliers.index, local_outliers['rec_error'], c='C3', marker='x', s=120, linewidth=3, label='local', edgecolors='w')

    # set axis labels
    xlabel = '$\\tau={}$' + str(mode+1) if mode == 0 else str(mode+1)
    axes[mode].set_xlabel(xlabel, fontsize=20)

    # set y-axis limits
    axes[mode].set_ylim([0.0, 1.1])

    # set x-axis ticks and labels
    axes[mode].set_xticks([int(plot_data.shape[0]/2)])
    axes[mode].set_xticklabels(['$x_{i}$'])

# set axis labels
axes[0].set_ylabel('Reconstruction Error $RE$', fontsize=20)

# init plot legend
handles, labels = axes[2].get_legend_handles_labels()

# plot legend
plt.legend(handles, labels, loc='center', fontsize=20, ncol=3, borderaxespad=0., bbox_to_anchor=(-6.5, 1., 9., .1))

# set grid plot layout
plt.grid(True)

### 5.4 Berechnung des Anomaly Scores

Die Mean-Divergenz und der Rekonstruktionsfehler einer Journalbuchung ermöglichen den Rückschluss, ob es sich bei der Buchung um eine gewöhnliche Buchung handelt bzw. ob die Buchung aus einer regulären Geschäftsaktivität resultiert. Um **globale** und **lokale Anomalien** zu ermitteln, soll nachfolgend jede Journalbuchung $x^i$ anhand der normalisierten Mean-Divergenz $MD_{\theta^*}^{\tau}$ und des Rekonstruktionsfehlers $RE_{\theta^*}^{\tau}$ bewertet werden. Hierzu verwenden wir den nachfolgenden Anomaly Score (AS), welcher beide Kriterien miteinander verknüpft:

\begin{equation}
AS^{\tau}(x^{i};\hat{x}^{i}) = \alpha \times RE_{\theta^*}^{\tau}(x^{i};\hat{x}^{i}) + (1-\alpha) \times MD_{\theta^*}^{\tau}(x^{i}),
\end{equation} 

wobei $x^{i}$ eine Journalbuchnug bezeichnet, $\theta^*$ ein gegebenes Set an Modellparametern und $\tau$ die Anzahl der Gaussians der Prior-Verteilung. Wir verknüpfen beide Eigenschaften additv und verwenden den Faktor $\alpha$ um beide Eigenschaften zu gewichten. Nachfolgend berechnen den Score pro Journalbuchung mit $\alpha=0.4$:

In [ ]:
# set alpha 
alpha = 0.4

# determine journal entry anomaly score
anomaly_score = alpha * rec_error_all_scaled + (1.0 - alpha) * mean_divergence_all_scaled

Abschliessend visualisieren wir den Anomaly Score $AS^{\tau}$ pro Journalbuchung $x^{i}$:

In [ ]:
# collect anomaly score, labels and cluster assignments
plot_data = pd.concat([pd.Series(anomaly_score, name='anomaly_score'), 
                       pd.Series(label, name='label'),                        
                       pd.Series(cluster_ids, name='cluster_id')],
                     axis=1)

# determine number of clusters
num_clusters = len(np.unique(cluster_ids))

# init sub-plots based on the number of modes
fig, axes = plt.subplots(1, num_clusters, sharey=True, figsize=(14, 10))

# iterate over distinct modes
for mode in range(0, num_clusters):

    # sample data fraction for faster visualization
    plot_data = plot_data.sample(frac=1.0)
    
    # collect features of current mode
    z_mode = plot_data[plot_data['cluster_id'] == mode]

    # seperate regular and anomalous journal entries
    regular_data = z_mode[z_mode['label'] == 'regular']
    global_outliers = z_mode[z_mode['label'] == 'global']
    local_outliers = z_mode[z_mode['label'] == 'local']

    # create train scatter plot of regular samples
    axes[mode].scatter(regular_data.index, regular_data['anomaly_score'], c='C0', marker='o', s=30, linewidth=0.3, label='regular', edgecolors='w')
    
    # create train scatter plot of global anomalies
    axes[mode].scatter(global_outliers.index, global_outliers['anomaly_score'], c='C1', marker='x', s=120, linewidth=3, label='global', edgecolors='w')
    
    # create train scatter plot of local anomalies
    axes[mode].scatter(local_outliers.index, local_outliers['anomaly_score'], c='C3', marker='x', s=120, linewidth=3, label='local', edgecolors='w')

    # set axis labels
    xlabel = '$\\tau={}$' + str(mode+1) if mode == 0 else str(mode+1)
    axes[mode].set_xlabel(xlabel, fontsize=20)

    # set y-axis limits
    axes[mode].set_ylim([0.0, 1.1])

    # set x-axis ticks and labels
    axes[mode].set_xticks([int(plot_data.shape[0]/2)])
    axes[mode].set_xticklabels(['$x_{i}$'])

# set axis labels
axes[0].set_ylabel('Anomaly Score $AS$', fontsize=20)

# init plot legend
handles, labels = axes[2].get_legend_handles_labels()

# plot legend
plt.legend(handles, labels, loc='center', fontsize=20, ncol=3, borderaxespad=0., bbox_to_anchor=(-6.5, 1., 9., .1))

# set grid plot layout
plt.grid(True)

Die Visualisierung zeigt, dass die regulären Journalbuchungen im Allgemeinen einen niedrigen Anomaly Score aufweisen. Zugleich weisen die im Buchungsstoff enthaltenen Anomalien einen vergleichsweise hohen Anomaly Score auf. Infolgedessen wird deutlich, dass sich anhand des Scorings 'globale' (orange) als auch 'lokale' (grün) Anomalien von regulären Tagebucheinträgen (blau) unterscheiden lassen. 

In [ ]:
ori_dataset['label'] = label
ori_dataset['tau'] = cluster_ids

Um beispielhaft einige der ermittelten Anomalien zu untersuchen, extrahieren wir nachfolgend die Journalbuchungen mit einem Anomaly Score >= 0.25 des Gaussian-Means $\tau=3$. Hierzu werden wir in einem ersten Schritt einen Zeitstempel des Datenextrakts für den Audit-Trail der Prüfung generieren: 

In [ ]:
timestamp = dt.datetime.utcnow().strftime("%Y-%m-%d_%H-%M-%S")

Anschliessend extrahieren wir die gefilterten **globalen Anomalien** als Excel-Datei zur weiteren substantiellen Prüfung:

In [ ]:
# inspect transactions exhibiting a reconstruction error >= 0.2
autoencoder_global_anomalies = ori_dataset[(anomaly_score >= 0.25) & (cluster_ids == 2)]

# specify the filename of the excel spreadsheet
filename = str(timestamp) + " - AAE_001_autoencoder_anomalies.xlsx"

# specify the target data directory of the excel spreadsheet
data_directory = os.path.join(results_directory, filename)

# extract the filtered transactions to excel
autoencoder_global_anomalies.to_excel(data_directory, header=True, index=False, sheet_name="Global_Anomalies", encoding="utf-8")

Darüber hinaus möchten wir beispielhaft weitere Anomalien untersuchen. Hierzu extrahieren wir die Journalbuchungen mit einem Anomaly-Score >= 0.4 des Gaussian-Means $\tau=2$. Hierzu erstellen wir in einem ersten Schritt wieder einen Zeitstempel für den Audit-Trail der Prüfung: 

In [ ]:
# inspect transactions exhibiting a anomaly_score >= 0.4 from the mode 2
autoencoder_local_anomalies = ori_dataset[(anomaly_score >= 0.4) & (cluster_ids == 1)]

# specify the filename of the excel spreadsheet
filename = str(timestamp) + " - AAE_002_autoencoder_anomalies.xlsx"

# specify the target data directory of the excel spreadsheet
data_directory = os.path.join(results_directory, filename)

# extract the filtered transactions to excel
autoencoder_local_anomalies.to_excel(data_directory, header=True, index=False, sheet_name="Local_Anomalies", encoding="utf-8")

## Lab Aufgaben:

Um Ihr wissen zu vertiefen empfehlen wir, die nachfolgenden Übungen zu bearbeiten:

**1. Trainieren und evaluieren Sie ein Adversarial Autoencoder Neural Network Model mit erhöhter Anzahl Means des Priors.**

> Die innerhalb des Notebooks vorgestellte Architektur führte zu einem guten Modell für die Erkennung von Anomalien. Prüfen Sie wie sich die Performance verändert, wenn Sie die Anzahl der Means der Mixture of Gaussian Prior-Verteilung erhöhen. Wie ändert sich die Fähigkeit des Modells Anomalien im Buchungsstoff zu erkennen? Lassen sich unterschiedliche Aussagen für globale bzw. lokale Anomalien treffen?

In [ ]:
# ***************************************************
# Sie können Ihre Lösung an dieser Stelle einfügen
# ***************************************************

**2. Evaluieren Sie unterschiedliche Gewichtungen $\alpha$ des Anomaly Scores.**

> Die innerhalb des Notebooks vorgestellte Gewichtung der normalisierten Mean-Divergence und des normalisierten Rekonstruktionsfehlers führte zu einem guten Ergebnis für die Erkennung von Anomalien. Prüfen Sie wie sich die Performance verändert, wenn Sie die Gewichtung der beiden Einflussfaktoren abänderen. Wie ändert sich die Fähigkeit des Modells Anomalien im Buchungsstoff zu erkennen? Lassen sich unterschiedliche Aussagen für globale bzw. lokale Anomalien treffen?

In [ ]:
# ***************************************************
# Sie können Ihre Lösung an dieser Stelle einfügen
# ***************************************************

## Lab Zusammenfassung

Diese Notebook umfasste eine schrittweise Einführung in **Entwurf, Implementierung, Training und Bewertung** eines auf einem neuronalen Netz basierenden Ansatzes zur Erkennung von Anomalien in Buchhaltungsdaten. Ein besonderer Augenmerk wurde hierbei auf das adversarial Lernen des Autoencoder Netzes gelegt. Die vorgestellten Code Beispiele und die Übungen können als Ausgangspunkt für für die Entwicklung und das Testen komplexerer Strategien zur Erkennung von Anomalien dienen.